In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.15) # Added Dropout 
        self.bn1 = nn.BatchNorm2d(16) # Added Batch Normalization
        self.bn2 = nn.BatchNorm2d(32) # Added second Batch Normalization
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) # Input 1 Output 16 here parameteres will be reduced
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1) # Input 16 Output 16
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1) # Input 16 Output 16
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) # Input 16 Output 32
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(32, 32, 3) # Input 32 Output 32
        self.conv6 = nn.Conv2d(32, 32, 3) # Input 32 Output 32
        self.conv7 = nn.Conv2d(32, 10, 3) # Input 32 Output 10

    def forward(self, x):
        x = self.pool1(self.drop1(self.bn1(F.relu(self.conv2(self.drop1(self.bn1(F.relu(self.conv1(x)))))))))
        x = self.pool2(self.drop1(self.bn2(F.relu(self.conv4(self.drop1(self.bn1(F.relu(self.conv3(x)))))))))
        x = self.drop1(self.bn2(F.relu(self.drop1(self.conv6(self.bn2(F.relu(self.conv5(x))))))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,320
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 32, 14, 14]           4,640
      BatchNorm2d-12           [-1, 32, 14, 14]              64
          Dropout-13           [-1, 32, 14, 14]               0
        MaxPool2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.RandomHorizontalFlip(),
                        # transforms.RandomResizedCrop(28),
                        transforms.RandomRotation(10, fill=(0,)),
                        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.9656, Accuracy: 6515/10000 (65%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.4954, Accuracy: 8361/10000 (84%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0944, Accuracy: 9747/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0976, Accuracy: 9728/10000 (97%)



loss=0.015064790844917297 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.79it/s]



Test set: Average loss: 0.0870, Accuracy: 9797/10000 (98%)

